# M_KK vs Yukawa: 2008 → 2024 Constraint Evolution

**The Question:** Why did M_KK = 3 TeV work in 2008 but not in 2024?

**The Answer:** MEG II tightened the μ→eγ bound by 100×, making the constraint 10× stricter.

In [ ]:
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams.update({'figure.dpi': 140, 'font.size': 11})

# Constraint constants
C_PAPER = 0.02              # Perez & Randall 2008 (MEGA)
C_MEGII = 0.001936          # MEG II 2024 (BR < 1.5e-13)

print(f"Constraint evolution:")
print(f"  2008: C = {C_PAPER:.4f}")
print(f"  2024: C = {C_MEGII:.6f}")
print(f"  Tightening: {C_PAPER/C_MEGII:.1f}×\n")

In [ ]:
# Load scan data
print("Loading scan results...")
files = sorted(glob.glob('scan_outputs/scan_shard_*_of_016.csv'))
df = pd.concat([pd.read_csv(f) for f in files], ignore_index=True)

# Compute which points pass each constraint
df['lfv_ratio_paper'] = df['lfv_ratio'] * (C_MEGII / C_PAPER)
df['passes_paper'] = df['perturbative'] & df['natural'] & (df['lfv_ratio_paper'] <= 1.0)
df['passes_megii'] = df['passes_all']

# Create subsets
df_pn = df[df.perturbative & df.natural].copy()
df_paper = df[df.passes_paper].copy()
df_megii = df[df.passes_megii].copy()

# Compute max neutrino Yukawa
y_N_cols = ['Y_N_bar_1', 'Y_N_bar_2', 'Y_N_bar_3']
df_pn['max_YN'] = df_pn[y_N_cols].abs().max(axis=1)
df_paper['max_YN'] = df_paper[y_N_cols].abs().max(axis=1)
df_megii['max_YN'] = df_megii[y_N_cols].abs().max(axis=1)

print(f"Loaded {len(df):,} scan points")
print(f"\nViable points:")
print(f"  2008 constraint: {len(df_paper):,}")
print(f"  MEG II 2024:     {len(df_megii):,}")
print(f"  Shrinkage:       {len(df_paper)/len(df_megii):.1f}×\n")

In [ ]:
# ============================================================================
# THE PLOT: M_KK vs Maximum Neutrino Yukawa
# ============================================================================

fig, ax = plt.subplots(1, 1, figsize=(14, 9))

# ===== STEP 1: Draw the constraint bounds =====
mkk_range = np.linspace(2, 20, 400)  # TeV

# The constraint is |(Y Y†)_12| ≤ C × (M_KK/3)²
# For roughly diagonal Y, this translates to Y_max ~ sqrt(C) × (M_KK/3)
y_bound_2008 = np.sqrt(C_PAPER) * (mkk_range / 3.0)
y_bound_2024 = np.sqrt(C_MEGII) * (mkk_range / 3.0)

# Fill the ALLOWED regions (below the curves)
ax.fill_between(mkk_range, 0, y_bound_2008,
                color='green', alpha=0.15,
                label='Allowed by 2008 bound')
ax.fill_between(mkk_range, 0, y_bound_2024,
                color='red', alpha=0.20,
                label='Allowed by MEG II 2024')

# Draw the boundary lines
ax.plot(mkk_range, y_bound_2008, 'g-', lw=4,
        label=f'2008 bound (C = {C_PAPER})', zorder=5)
ax.plot(mkk_range, y_bound_2024, 'r-', lw=4,
        label=f'MEG II 2024 (C = {C_MEGII:.4f})', zorder=5)

# ===== STEP 2: Scatter the actual scan points =====
np.random.seed(42)

# Gray: excluded by both
excluded = df_pn[~df_pn.index.isin(df_paper.index)]
sample_ex = excluded.sample(min(3000, len(excluded)))
ax.scatter(sample_ex['Lambda_IR']/1000, sample_ex['max_YN'],
           c='#cccccc', s=5, alpha=0.3, rasterized=True,
           label=f'Excluded ({len(excluded):,} points)', zorder=1)

# Green: passed 2008 but fails MEG II (LOST SPACE)
lost = df_paper[~df_paper.index.isin(df_megii.index)]
sample_lost = lost.sample(min(2000, len(lost)))
ax.scatter(sample_lost['Lambda_IR']/1000, sample_lost['max_YN'],
           c='#27ae60', s=20, alpha=0.7, rasterized=True,
           edgecolors='darkgreen', linewidths=0.5,
           label=f'Lost to MEG II ({len(lost):,} points)', zorder=3)

# Red: passes MEG II (still viable)
ax.scatter(df_megii['Lambda_IR']/1000, df_megii['max_YN'],
           c='#e74c3c', s=25, alpha=0.85, rasterized=True,
           edgecolors='darkred', linewidths=0.6,
           label=f'MEG II viable ({len(df_megii):,} points)', zorder=4)

# ===== STEP 3: Reference lines and annotations =====
# Naturalness target
ax.axhline(1.0, color='goldenrod', ls='--', lw=3, alpha=0.9, zorder=2,
           label=r'O(1) naturalness target')
ax.axhspan(0.3, 3.0, color='gold', alpha=0.05, zorder=0)

# Mark M_KK = 3 TeV (2008 benchmark)
ax.axvline(3.0, color='blue', ls=':', lw=3, alpha=0.8, zorder=2)
ax.text(3.1, 3.4, 'M_KK = 3 TeV\n(2008 benchmark)',
        fontsize=13, color='blue', weight='bold',
        bbox=dict(boxstyle='round,pad=0.5', facecolor='white',
                  edgecolor='blue', linewidth=2, alpha=0.95))

# Mark where MEG II allows Y ~ 1
mkk_for_natural = 3.0 / np.sqrt(C_MEGII)
ax.axvline(mkk_for_natural, color='darkred', ls=':', lw=3, alpha=0.8, zorder=2)
ax.text(mkk_for_natural - 10, 2.2,
        f'MEG II allows Y~1\nat M_KK ≈ {mkk_for_natural:.0f} TeV!',
        fontsize=12, color='darkred', weight='bold',
        bbox=dict(boxstyle='round,pad=0.5', facecolor='white',
                  edgecolor='darkred', linewidth=2, alpha=0.95))

# Key insight box
insight_text = (
    'Key Insight:\n'
    f'• 2008: M_KK = 3 TeV → Y_N ~ {np.sqrt(C_PAPER):.2f} ✓\n'
    f'• 2024: M_KK = 3 TeV → Y_N ~ {np.sqrt(C_MEGII):.2f} ✗\n'
    f'• Need M_KK ~ {mkk_for_natural:.0f} TeV for Y_N ~ 1\n'
    f'  (that\'s {mkk_for_natural/3:.1f}× higher!)'
)
ax.text(0.98, 0.03, insight_text,
        transform=ax.transAxes, fontsize=11, family='monospace',
        verticalalignment='bottom', horizontalalignment='right',
        bbox=dict(boxstyle='round,pad=0.8', facecolor='wheat',
                  edgecolor='black', linewidth=1.5, alpha=0.9))

# ===== STEP 4: Labels and styling =====
ax.set_xlabel(r'$M_{KK} = \Lambda_{IR}$ (TeV)', fontsize=16, weight='bold')
ax.set_ylabel(r'max$(|\bar{Y}_N|)$ — Maximum Neutrino Yukawa',
              fontsize=16, weight='bold')
ax.set_title('Why 3 TeV Worked in 2008 But Not in 2024:\nThe MEG II Constraint Evolution',
             fontsize=17, weight='bold', pad=20)

ax.set_xlim(2, 20)
ax.set_ylim(0, 3.8)
ax.legend(fontsize=11, loc='upper left', framealpha=0.98,
          edgecolor='black', fancybox=False, shadow=True)
ax.grid(True, alpha=0.3, linewidth=0.9)

plt.tight_layout()
plt.savefig('mkk_vs_yukawa_2008_vs_2024.png', dpi=150, bbox_inches='tight')
plt.savefig('mkk_vs_yukawa_2008_vs_2024.pdf', bbox_inches='tight')

print(f"\n{'='*75}")
print('SAVED: mkk_vs_yukawa_2008_vs_2024.png & .pdf')
print(f"{'='*75}\n")

plt.show()

# Print summary
print(f"\n{'='*75}")
print('SUMMARY: CONSTRAINT EVOLUTION 2008 → 2024')
print(f"{'='*75}")
print(f"\nBranching ratio limits:")
print(f"  MEGA (2008):  BR(μ→eγ) < 1.2e-11")
print(f"  MEG II (2024): BR(μ→eγ) < 1.5e-13  (100× better!)")
print(f"\nEffective constraint:")
print(f"  2008: C = {C_PAPER:.4f}")
print(f"  2024: C = {C_MEGII:.6f}  ({C_PAPER/C_MEGII:.1f}× tighter)")
print(f"\nParameter space impact:")
print(f"  Viable points: {len(df_paper):,} → {len(df_megii):,}  ({len(df_paper)/len(df_megii):.1f}× reduction)")
print(f"  Lost points:   {len(lost):,} ({100*len(lost)/len(df_paper):.1f}% of 2008 viable)")
print(f"\nM_KK requirements for Y_N ~ 1:")
print(f"  2008:  ~7 TeV")
print(f"  2024:  ~{mkk_for_natural:.0f} TeV  ({mkk_for_natural/7:.1f}× higher)")
print(f"\n{'='*75}")
print('CONCLUSION: MEG II requires M_KK ≥ 10-15 TeV for partially natural Yukawas')
print(f"{'='*75}\n")